# Linear Programming

Optimisation problems. Source: [https://realpython.com/linear-programming-python/](https://realpython.com/linear-programming-python/)

- linear programming
- mixed integer linear programming

SciPy is straightforward to set up. Once you install it, you’ll have everything you need to start. Its subpackage scipy.optimize can be used for both linear and nonlinear optimization.

PuLP allows you to choose solvers and formulate problems in a more natural way. The default solver used by PuLP is the COIN-OR Branch and Cut Solver (CBC). It’s connected to the COIN-OR Linear Programming Solver (CLP) for linear relaxations and the COIN-OR Cut Generator Library (CGL) for cuts generation.

Another great open source solver is the GNU Linear Programming Kit (GLPK). Some well-known and very powerful commercial and proprietary solutions are Gurobi, CPLEX, and XPRESS.

## Installation

```sh
python -m pip install -U "scipy==1.4.*" "pulp==2.1"
```

If problems with `pulp` e.g. `AttributeError: module 'enum' has no attribute 'IntFlag'`, then `pip uninstall -y enum34` and try again to install `pulp`.

You might need to run pulptest or sudo pulptest to enable the default solvers for PuLP, especially if you’re using Linux or Mac: `pulptest`.

On Debian and Ubuntu, use apt to install GLPK:

```sh
sudo apt install glpk-utils
conda install -c conda-forge glpk
```

After completing the installation, you can check the version of GLPK: `glpsol --version`

# Using SciPy

## Example 1

Maximize: $z = x + 2y$

\begin{equation}
    \begin{cases}
      2x + y \le 20\\
      -4x + 5y \le 10\\
      -x + 2y \ge -2\\
      -x + 5y = 15\\
      x \ge 0\\
      y \ge 0\\
    \end{cases}\,
\end{equation}

`linprog()` solves only minimization (not maximization) problems and doesn’t allow inequality constraints with the greater than or equal to sign (≥).

Workarounds:

- Instead of maximizing $z = x + 2y$, you can minimize its negative $−z = −x − 2y$.
- Instead of having the greater than or equal to sign, you can multiply the yellow inequality by −1 and get the opposite less than or equal to sign (≤).

\begin{equation}
    \begin{cases}
      2x + y \le 20\\
      -4x + 5y \le 10\\
      x - 2y \le 2\\
      -x + 5y = 15\\
      x \ge 0\\
      y \ge 0\\
    \end{cases}\,
\end{equation}

Minimize: $-z = -x -2y$


In [1]:
from scipy.optimize import linprog

In [2]:
# objective function that we want to MINIMIZE
obj = [-1, -2]
#      ─┬  ─┬
#       │   └┤ Coefficient for y
#       └────┤ Coefficient for x

In [3]:
# inequalities part of the system of equations
lhs_ineq = [[ 2,  1],  # Red constraint left side
            [-4,  5],  # Blue constraint left side
            [ 1, -2]]  # Yellow constraint left side

rhs_ineq = [20,  # Red constraint right side
            10,  # Blue constraint right side
             2]  # Yellow constraint right side

In [4]:
# equations part of the system of equations
lhs_eq = [[-1, 5]]  # Green constraint left side
rhs_eq = [15]       # Green constraint right side

- **`obj`** holds the coefficients from the objective function.
- **`lhs_ineq`** holds the left-side coefficients from the inequality (red, blue, and yellow) constraints.
- **`rhs_ineq`** holds the right-side coefficients from the inequality (red, blue, and yellow) constraints.
- **`lhs_eq`** holds the left-side coefficients from the equality (green) constraint.
- **`rhs_eq`** holds the right-side coefficients from the equality (green) constraint.

The next step is to define the bounds for each variable in the same order as the coefficients. In this case, they’re both between zero and positive infinity:

This statement is redundant because `linprog()` takes these bounds (zero to positive infinity) by default.

In [5]:
bnd = [(0, float("inf")),  # Bounds of x
       (0, float("inf"))]  # Bounds of y

it’s time to optimize and solve your problem of interest

In [6]:
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
              A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
              method="revised simplex")

In [7]:
opt

     con: array([0.])
     fun: -16.818181818181817
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([ 0.        , 18.18181818,  3.36363636])
  status: 0
 success: True
       x: array([7.72727273, 4.54545455])

- The parameter `c` refers to the coefficients from the objective function. 
- `A_ub` and `b_ub` are related to the coefficients from the left and right sides of the inequality constraints, respectively.
- Similarly, `A_eq` and `b_eq` refer to equality constraints. 
- You can use `bounds` to provide the lower and upper bounds on the decision variables.

You can use the parameter `method` to define the linear programming method that you want to use. There are three options:

- method="interior-point" selects the interior-point method. This option is set by default.
- method="revised simplex" selects the revised two-phase simplex method.
- method="simplex" selects the legacy two-phase simplex method.

`linprog()` returns a data structure with these attributes:

- `.con` is the equality constraints residuals.
- `.fun` is the objective function value at the optimum (if found).
- `.message` is the status of the solution.
- `.nit` is the number of iterations needed to finish the calculation.
- `.slack` is the values of the slack variables, or the differences between the values of the left and right sides of the constraints.
- `.status` is an integer between 0 and 4 that shows the status of the solution, such as 0 for when the optimal solution has been found.
- `.success` is a Boolean that shows whether the optimal solution has been found.
- `.x` is a NumPy array holding the optimal values of the decision variables.

You can access these values separately e.g. `opt.fun` and so on.

## Example 2

Resource allocation problem.

Say that a factory produces four different products, and that the daily produced amount of the first product is $x_1$, the amount produced of the second product is $x_2$, and so on. The goal is to determine the profit-maximizing daily production amount for each product, bearing in mind the following conditions:

1. The profit per unit of product is \\$20, \\$12, \\$40, and \\$25 for the first, second, third, and fourth product, respectively.
2. Due to manpower constraints, the total number of units produced per day can’t exceed fifty.
3. For each unit of the first product, three units of the raw material A are consumed. Each unit of the second product requires two units of the raw material A and one unit of the raw material B. Each unit of the third product needs one unit of A and two units of B. Finally, each unit of the fourth product requires three units of B.
4. Due to the transportation and storage constraints, the factory can consume up to one hundred units of the raw material A and ninety units of B per day.

Maximize profit: $max -[ 20 x_1 + 12 x_2 + 40 x_3 + 25 x_4 ]$

\begin{equation}
    \begin{cases}
      x_1 + x_2 + x_3 + x_4 \le 50 (manpower)\\
      3 x_1 + 2 x_2 + x_3 \le 100 (material A)\\
      x_2 + 2 x_3 + 3 x_4 \le 90 (material B)\\
      x_1 \ge 0\\
      x_2 \ge 0\\
      x_3 \ge 0\\
      x_4 \ge 0\\
    \end{cases}\,
\end{equation}


In [8]:
obj = [-20, -12, -40, -25]

In [9]:
lhs_ineq = [[1, 1, 1, 1],  # Manpower
            [3, 2, 1, 0],  # Material A
            [0, 1, 2, 3]]  # Material B

In [10]:
rhs_ineq = [ 50,  # Manpower
            100,  # Material A
             90]  # Material B

In [11]:
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
              method="revised simplex")

In [12]:
opt

     con: array([], dtype=float64)
     fun: -1900.0
 message: 'Optimization terminated successfully.'
     nit: 2
   slack: array([ 0., 40.,  0.])
  status: 0
 success: True
       x: array([ 5.,  0., 45.,  0.])

The result tells you that the maximal profit is $1900$ and corresponds to $x_1 = 5$ and $x_3 = 45$. It’s not profitable to produce the second and fourth products under the given conditions.

1. The third product brings the largest profit per unit, so the factory will produce it the most.

2. The first slack is 0, which means that the values of the left and right sides of the manpower (first) constraint are the same. The factory produces 50 units per day, and that’s its full capacity.

3. The second slack is 40 because the factory consumes 60 units of raw material A (15 units for the first product plus 45 for the third) out of a potential 100 units.

4. The third slack is 0, which means that the factory consumes all 90 units of the raw material B. This entire amount is consumed for the third product. That’s why the factory can’t produce the second or fourth product at all and can’t produce more than 45 units of the third product. It lacks the raw material B.

# Using PuLP

PuLP has a more convenient linear programming API than SciPy. You don’t have to mathematically modify your problem or use vectors and matrices. Everything is cleaner and less prone to errors.

## Example 1

Maximize: $z = x + 2y$

\begin{equation}
    \begin{cases}
      2x + y \le 20\\
      -4x + 5y \le 10\\
      -x + 2y \ge -2\\
      -x + 5y = 15\\
      x \ge 0\\
      y \ge 0\\
    \end{cases}\,
\end{equation}

In [13]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [14]:
# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

Once that you have the model, you can define the decision variables as instances of the `LpVariable` class.

In [15]:
# Initialize the decision variables
x = LpVariable(name="x", lowBound=0)
y = LpVariable(name="y", lowBound=0)

You need to provide a lower bound with `lowBound=0` because the default value is negative infinity. The parameter `upBound` defines the upper bound, but you can omit it here because it defaults to positive infinity.

The optional parameter cat defines the category of a decision variable. If you’re working with continuous variables, then you can use the default value `"Continuous"`.

In [16]:
# example PuLP expression
expression = 2 * x + 4 * y
type(expression)

pulp.pulp.LpAffineExpression

In [17]:
# example PuLP expression
constraint = 2 * x + 4 * y >= 8
type(constraint)

pulp.pulp.LpConstraint

In [18]:
# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

In [19]:
# Add the objective function to the model
obj_func = x + 2 * y
model += obj_func

In [20]:
# You can now see the full definition of this model:
model

small-problem:
MAXIMIZE
1*x + 2*y + 0
SUBJECT TO
red_constraint: 2 x + y <= 20

blue_constraint: 4 x - 5 y >= -10

yellow_constraint: - x + 2 y >= -2

green_constraint: - x + 5 y = 15

VARIABLES
x Continuous
y Continuous

In [21]:
# Solve the problem
status = model.solve()

`.solve()` calls the underlying solver, modifies the model object, and returns the integer status of the solution, which will be 1 if the optimum is found. 

In [22]:
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [23]:
print(f"objective: {model.objective.value()}")

objective: 16.8181817


In [24]:
for var in model.variables():
    print(f"{var.name}: {var.value()}")

x: 7.7272727
y: 4.5454545


In [25]:
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

red_constraint: -9.99999993922529e-08
blue_constraint: 18.181818300000003
yellow_constraint: 3.3636362999999996
green_constraint: -2.0000000233721948e-07


`model.objective` holds the value of the objective function, `model.constraints` contains the values of the slack variables, and the objects `x` and `y` have the optimal values of the decision variables.

You can see which solver was used by calling `.solver`:

In [26]:
model.solver

If you want to run a different solver, then you can specify it as an argument of .solve()

In [27]:
from pulp import GLPK

In [28]:
# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

# Initialize the decision variables
x = LpVariable(name="x", lowBound=0)
y = LpVariable(name="y", lowBound=0)

# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

# Add the objective function to the model
model += lpSum([x, 2 * y])

# Solve the problem
status = model.solve(solver=GLPK(msg=False))

In [29]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 16.81817
x: 7.72727
y: 4.54545
red_constraint: -1.0000000000509601e-05
blue_constraint: 18.181830000000005
yellow_constraint: 3.3636299999999997
green_constraint: -2.000000000279556e-05


In [30]:
model.solver

**You can also use PuLP to solve mixed-integer linear programming problems.** To define an integer or binary variable, just pass `cat="Integer"` or `cat="Binary"` to `LpVariable`. Everything else remains the same:

In [31]:
# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

# Initialize the decision variables: x is integer, y is continuous
x = LpVariable(name="x", lowBound=0, cat="Integer")
y = LpVariable(name="y", lowBound=0)

# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

# Add the objective function to the model
model += lpSum([x, 2 * y])

# Solve the problem
status = model.solve()

In [32]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 15.8
x: 7.0
y: 4.4
red_constraint: -1.5999999999999996
blue_constraint: 16.0
yellow_constraint: 3.8000000000000007
green_constraint: 0.0


In [33]:
model.solver

Now `x` is an integer, as specified in the model. (Technically it holds a float value with zero after the decimal point.) This fact changes the whole solution.

## Example 2

Maximize profit: $max -[ 20 x_1 + 12 x_2 + 40 x_3 + 25 x_4 ]$

\begin{equation}
    \begin{cases}
      x_1 + x_2 + x_3 + x_4 \le 50 (manpower)\\
      3 x_1 + 2 x_2 + x_3 \le 100 (material A)\\
      x_2 + 2 x_3 + 3 x_4 \le 90 (material B)\\
      x_1 \ge 0\\
      x_2 \ge 0\\
      x_3 \ge 0\\
      x_4 \ge 0\\
    \end{cases}\,
\end{equation}

In [34]:
# Define the model
model = LpProblem(name="resource-allocation", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}

# Add constraints
model += (lpSum(x.values()) <= 50, "manpower")
model += (3 * x[1] + 2 * x[2] + x[3] <= 100, "material_a")
model += (x[2] + 2 * x[3] + 3 * x[4] <= 90, "material_b")

# Set the objective
model += 20 * x[1] + 12 * x[2] + 40 * x[3] + 25 * x[4]

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 1900.0
x1: 5.0
x2: 0.0
x3: 45.0
x4: 0.0
manpower: 0.0
material_a: -40.0
material_b: 0.0


Let’s make this problem more complicated and interesting. Say the factory can’t produce the first and third products in parallel due to a machinery issue. What’s the most profitable solution in this case?

Now you have another logical constraint: if $x_1$ is positive, then $x_3$ must be zero and vice versa. This is where **binary decision variables** are very useful. 

In [35]:
model = LpProblem(name="resource-allocation", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}
y = {i: LpVariable(name=f"y{i}", cat="Binary") for i in (1, 3)}

# Add constraints
model += (lpSum(x.values()) <= 50, "manpower")
model += (3 * x[1] + 2 * x[2] + x[3] <= 100, "material_a")
model += (x[2] + 2 * x[3] + 3 * x[4] <= 90, "material_b")

M = 100
model += (x[1] <= y[1] * M, "x1_constraint")
model += (x[3] <= y[3] * M, "x3_constraint")
model += (y[1] + y[3] <= 1, "y_constraint")

# Set objective
model += 20 * x[1] + 12 * x[2] + 40 * x[3] + 25 * x[4]

# Solve the optimization problem
status = model.solve()

print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 1800.0
x1: 0.0
x2: 0.0
x3: 45.0
x4: 0.0
y1: 0.0
y3: 1.0
manpower: -5.0
material_a: -55.0
material_b: 0.0
x1_constraint: 0.0
x3_constraint: -55.0
y_constraint: 0.0


It turns out that the optimal approach is to exclude the first product and to produce only the third one.

# Linear Programming 

- Integer programming [https://towardsdatascience.com/integer-programming-in-python-1cbdfa240df2](https://towardsdatascience.com/integer-programming-in-python-1cbdfa240df2)
- Knapsack Problem